<a href="https://colab.research.google.com/github/Sruthi-sk/MultiDeepRLNeuroGambling/blob/main/RL_single_agent_environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Useful Links:
*   https://deeplearning.neuromatch.io/projects/ReinforcementLearning/human_rl.html
*   https://docs.ray.io/en/master/rllib/index.html 
* https://docs.ray.io/en/master/rllib/core-concepts.html 
* https://medium.com/distributed-computing-with-ray/anatomy-of-a-custom-environment-for-rllib-327157f269e5
* https://www.youtube.com/watch?v=Mut_u40Sqz4&t=1s


###Installing dependencies


In [ ]:
# !pip install ray[rllib]=="1.13.0"

In [ ]:
# !pip install "ray[rllib]" torch

###Testing if works on colab

###Creating the environment

RL Single Agent

State Space:

*   fMRI state
*   balance
*   previous Win/Loss/Neutral


Action Space:
* Bet High/Low (50-50 chance)
* ~~Stay Neutral~~

Transition Space (depends on rules):
* Previous time-weighted fMRI reading is used as input

Default HCP Scenario:
* Win +1
* Loss -0.5


Episode length?
100

okay, so the environment should be modified to reflect this 3rd condition that you draw? So for now just a player vs. casino with 50% chance of winning

## Environment based on this tutorial
(https://medium.com/distributed-computing-with-ray/anatomy-of-a-custom-environment-for-rllib-327157f269e5)

https://www.youtube.com/watch?v=bD6V3rcr_54 
https://github.com/nicknochnack/OpenAI-Reinforcement-Learning-with-Custom-Environment/blob/main/OpenAI%20Custom%20Environment%20Reinforcement%20Learning.ipynb 

# Task Description

---



"""Task description from HCP:
Participants play a card guessing game where they are asked to guess the number on amystery card (represented by a “?”) in order to win or lose money. Participants are told that
potential card numbers range from 1-9 and to indicate if they think the mystery card number is
more or less than 5 by pressing one of two buttons on the response box. Feedback is the
number on the card (generated by the program as a function of whether the trial was a reward,
loss or neutral trial) and either: 1) a green up arrow with “$1” for reward trials, 2) a red down
arrow next to -$0.50 for loss trials; or 3) the number 5 and a gray double headed arrow for
neutral trials. The “?” is presented for up to 1500 ms (if the participant responds before 1500
ms, a fixation cross is displayed for the remaining time), following by feedback for 1000 ms.
There is a 1000 ms ITI with a “+” presented on the screen. The task is presented in blocks of 8
trials that are either mostly reward (6 reward trials pseudo randomly interleaved with either 1 neutral and 1 loss trial, 
2 neutral trials, or 2 loss trials) or mostly loss (6 loss trials pseudorandomly interleaved with either 1 neutral and 1 reward trial, 2 neutral trials, or 2 reward trials).
In each of the two runs, there are 2 mostly reward and 2 mostly loss blocks, interleaved with 4
fixation blocks (15 seconds each).
"""



# Define Env

In [ ]:
#Import libs
import math
from typing import Optional, Union

import numpy as np
import random

import gym
from gym import logger, spaces
# from gym.envs.classic_control import utils
from gym.error import DependencyNotInstalled
# from gym.utils.renderer import Renderer
from gym.utils import seeding

In [ ]:

#GamblingTask
# HM_EPISODES = 50
# epsilon = 0.9
# EPS_DECAY = 0.9998
# LEARNING_RATE = 0.1
# DISCOUNT = 0.95
# timestep_limit=1000
# reward_threshold=1.0
# nondeterministic = True

# #Rewards:
# win = 1
# loss = -1
# neut = 0
#Number of trials(steps) per episode:

class GamblingTask(gym.Env):

  metadata = {
    "render.modes": ["human"]  #rendering text to the terminal
  }

  def __init__(self):
    #Action space:
    self.action_space = gym.spaces.Discrete(3)
    self.observation_space = gym.spaces.Box(low=np.array([0]), high=np.array([200]))
    self.state = 100
    # self.reset()
    self.max_steps = 20 # max no of times they can bet

  def reset(self):
      # Reset money balance
      self.state = 100     # balance
      # Reset max no of bets
      self.max_steps = 20 
      #   self.done = False
      return self.state


  def step(self, action,coin_toss):
    #let coin toss be 0 or 2

    # if self.done:
    #   print("Episode done")
    # elif self.trial_count == self.max_step:
    #   self.done = True;
    # else:
    #   assert self.action_space.contains(action)
    if action==coin_toss:
        reward =5 
    elif action==1:
        reward=0
    else: 
        reward = -5
    self.state += reward 
    self.max_steps -=1
    
    if self.max_steps <= 0: 
            done = True
    else:
            done = False

    info = {}

    # try:
    #   assert self.observation_space.contains(self.state)
    # except AssertionError:
    #       print("INVALID STATE", self.state)
    return self.state, reward, done, info


  def render (self, mode="human"):
    print(self.state,self.max_steps)

  # def seed (self, seed=None):
  #   self.np_random, seed = seeding.np_random(seed)
  #   return [seed]

  def close(self):
    pass

    

In [ ]:
env=GamblingTask()

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:
print("Action Space: {}".format(env.action_space))
print("State space: {}".format(env.observation_space))
print(env.observation_space.sample())


Action Space: Discrete(3)
State space: Box(0.0, 200.0, (1,), float32)
[108.70369]


In [ ]:
states=env.observation_space.shape
actions=env.action_space.n

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    state=env.reset()
    done = False
    score = 0 
    while not done:
        # env.render()
        coin_toss = random.choice([0,2])
        action = env.action_space.sample() #bet_low = 0, bet_none=1, bet_high = 2
        # print("actual ",coin_toss,"action ",action)
        n_state, reward, done, info = env.step(action,coin_toss)
        score+=reward
        # print("reward ",reward, "score ",score)
        # we have to return current reward (win/lose)
    print('Episode:{} Score:{}'.format(episode, score))
    env.close()

Episode:1 Score:0
Episode:2 Score:10
Episode:3 Score:15
Episode:4 Score:-5
Episode:5 Score:10


##define DL model which takes states as input, action as output

Convert to Pytorch

In [ ]:

#GamblingTask
# HM_EPISODES = 50
# epsilon = 0.9
# EPS_DECAY = 0.9998
# LEARNING_RATE = 0.1
# DISCOUNT = 0.95
# timestep_limit=1000
# reward_threshold=1.0
# nondeterministic = True

# #Rewards:
# win = 1
# loss = -1
# neut = 0
#Number of trials(steps) per episode:

class GamblingTask(gym.Env):

  metadata = {
    "render.modes": ["human"]  #rendering text to the terminal
  }

  def __init__(self):
    #Action space:
    self.action_space = gym.spaces.Discrete(3)
    self.observation_space = gym.spaces.Box(low=np.array([0]), high=np.array([200]))
    self.state = 100
    # self.reset()
    self.max_steps = 20 # max no of times they can bet

  def reset(self):
      # Reset money balance
      self.state = 100     # balance
      # Reset max no of bets
      self.max_steps = 20 
      #   self.done = False
      return self.state


  def step(self, action):
    #let coin toss be 0 or 2

    # if self.done:
    #   print("Episode done")
    # elif self.trial_count == self.max_step:
    #   self.done = True;
    # else:
    #   assert self.action_space.contains(action)
    coin_toss = random.choice([0,2])
    if action==coin_toss:
        reward =5 
    elif action==1:
        reward=0
    else: 
        reward = -5
    self.state += reward 
    self.max_steps -=1
    
    if self.max_steps <= 0: 
            done = True
    else:
            done = False

    info = {}

    # try:
    #   assert self.observation_space.contains(self.state)
    # except AssertionError:
    #       print("INVALID STATE", self.state)
    return self.state, reward, done, info


  def render (self, mode="human"):
    print(self.state,self.max_steps)

  # def seed (self, seed=None):
  #   self.np_random, seed = seeding.np_random(seed)
  #   return [seed]

  def close(self):
    pass


In [ ]:
env=GamblingTask()

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:
# Define model here

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
states = env.observation_space.shape
actions = env.action_space.n
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model
model = build_model(states, actions)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 24)                48        
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [ ]:
del model

# Build Agent

In [ ]:
!pip install keras-rl2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52 kB 772 kB/s 


In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,       
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=500, visualize=False, verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training for 500 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 12:05 - reward: -5.0000

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


  498/10000 [>.............................] - ETA: 1:19 - reward: -0.1506done, took 4.247 seconds


In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 0.000, steps: 20
Episode 2: reward: -20.000, steps: 20
Episode 3: reward: 40.000, steps: 20
Episode 4: reward: 0.000, steps: 20
Episode 5: reward: 20.000, steps: 20
Episode 6: reward: 30.000, steps: 20
Episode 7: reward: 10.000, steps: 20
Episode 8: reward: -30.000, steps: 20
Episode 9: reward: 0.000, steps: 20
Episode 10: reward: -20.000, steps: 20
Episode 11: reward: 10.000, steps: 20
Episode 12: reward: 10.000, steps: 20
Episode 13: reward: 20.000, steps: 20
Episode 14: reward: 20.000, steps: 20
Episode 15: reward: 10.000, steps: 20
Episode 16: reward: -10.000, steps: 20
Episode 17: reward: 30.000, steps: 20
Episode 18: reward: 10.000, steps: 20
Episode 19: reward: 10.000, steps: 20
Episode 20: reward: 10.000, steps: 20
Episode 21: reward: 0.000, steps: 20
Episode 22: reward: 20.000, steps: 20
Episode 23: reward: 10.000, steps: 20
Episode 24: reward: 0.000, steps: 20
Episode 25: reward: 30.000, steps: 20
Episode 26: reward: -10.000, st

In [ ]:
results = dqn.test(env, nb_episodes=150, visualize=False)
print(np.mean(results.history['episode_reward']))

Testing for 150 episodes ...
Episode 1: reward: 20.000, steps: 20
Episode 2: reward: -10.000, steps: 20
Episode 3: reward: 0.000, steps: 20
Episode 4: reward: 0.000, steps: 20
Episode 5: reward: 10.000, steps: 20
Episode 6: reward: 20.000, steps: 20
Episode 7: reward: 20.000, steps: 20
Episode 8: reward: 20.000, steps: 20
Episode 9: reward: 0.000, steps: 20
Episode 10: reward: 0.000, steps: 20
Episode 11: reward: -50.000, steps: 20
Episode 12: reward: -10.000, steps: 20
Episode 13: reward: -20.000, steps: 20
Episode 14: reward: 20.000, steps: 20
Episode 15: reward: 0.000, steps: 20
Episode 16: reward: -10.000, steps: 20
Episode 17: reward: -20.000, steps: 20
Episode 18: reward: 0.000, steps: 20
Episode 19: reward: 0.000, steps: 20
Episode 20: reward: -20.000, steps: 20
Episode 21: reward: -20.000, steps: 20
Episode 22: reward: 10.000, steps: 20
Episode 23: reward: -20.000, steps: 20
Episode 24: reward: 20.000, steps: 20
Episode 25: reward: 20.000, steps: 20
Episode 26: reward: -30.000,